In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score

# Загрузка данных
train_path = 'nlp-getting-started/train.csv'
test_path = 'nlp-getting-started/test.csv'
ss_path = 'nlp-getting-started/sample_submission.csv'

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)
ss_data = pd.read_csv(ss_path)

target_variable = 'target'

# Заполнение пропусков
train_data['location'] = train_data['location'].fillna('Unknown')
train_data['full_text'] = train_data['keyword'] + ' ' + train_data['text']
train_data['full_text'] = train_data['full_text'].astype(str)

test_data['location'] = test_data['location'].fillna('Unknown')
test_data['full_text'] = test_data['keyword'] + ' ' + test_data['text']
test_data['full_text'] = test_data['full_text'].astype(str)

features = ['full_text', 'location']
X = train_data[features]
y = train_data[target_variable]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

categorical_features = ['location']
text_features = ['full_text']

from catboost import CatBoostClassifier

model = CatBoostClassifier(
    iterations=1500,            # default=500
    learning_rate=0.1,          # default=0.1
    depth=6,                    # default=6
    loss_function='Logloss',    # default='Logloss'
    verbose=50,                 # default=50
    task_type='GPU',            # default='GPU'
)

print(X_test)

model.fit(
    X=X_train, 
    y=y_train, 
    cat_features=categorical_features, 
    text_features=text_features, 
)

y_pred = model.predict(X_test)


from sklearn.metrics import accuracy_score

print("Accuracy:", accuracy_score(y_test, y_pred))

# 0.7984241628365069 iters=500
# 0.8102429415627052 iters=1500

<class 'pandas.core.frame.DataFrame'>
Index: 6090 entries, 6234 to 7569
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   full_text  6090 non-null   object
 1   location   6090 non-null   object
dtypes: object(2)
memory usage: 142.7+ KB
None
                                              full_text  \
4863  mass%20murderer @TheEconomist Step one: get th...   
1370  bush%20fires Ted Cruz fires back at Jeb &amp; ...   
3521  eyewitness How ÛÏLittle BoyÛ Affected the P...   
178   ambulance AMBULANCE SPRINTER AUTOMATIC FRONTLI...   
5859  ruin 'Cause you play me like a symphony play m...   
...                                                 ...   
6939  trouble @PrinceofFencing frickin summer and it...   
2074  dead @emmerdale is Ross really dead?? #AskCharley   
3186  emergency%20plan b/c it costs less to have sic...   
4297  hellfire @HellFire_eV @JackPERU1 then I do thi...   
2344  demolition http://t.co/D3cTNI8Qit #Dem

In [20]:
# Предсказания на тестовом наборе
test_preds = model.predict(test_data[features])

# Формирование файла для отправки
submission = pd.DataFrame({
    'id': test_data['id'],
    'target': test_preds
})

submission.to_csv('NLPwDT_03_pred_catboost.csv', index=False)